<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Treinamento_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configurações Iniciais 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

## Lendo a Base

In [3]:
# Base1 Com StopWords
PATH_BASE1 = "/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv"

# Base2 Sem StopWords
PATH_BASE2 = "/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv"

# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"

In [4]:
#trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA, index_col=0)
trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA, index_col=0)

In [5]:
# Agrupa dados por label
trainDF.groupby('label').count()

,text
label,
0,2443
1,2443


In [53]:
# Pega a média de caracteres dos tweets de toda a base
average_len = int(trainDF['text'].apply(lambda x: len(str(x).split(' '))).mean())
average_len

10

In [54]:
# Pega a média de caracteres dos tweets de toda a base
max_len = int(trainDF['text'].apply(lambda x: len(str(x).split(' '))).max())
max_len

25

In [55]:
palavras_unicas = list ( trainDF['text'].str.split(' ', expand=True).stack().unique() ) 
quantidade_palavras_unicas = len( palavras_unicas )
quantidade_palavras_unicas

9144

#TF-IDF (NÃO CONSEGUIMOS USAR NO MODELO CNN)

In [56]:
# word level tf-idf
# td-idf gera matriz com 9140 colunas (4 palavras estão sendo perdidas ao usar o regex \w{1,})
# tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features= quantidade_palavras_unicas)
# X = tfidf_vect.fit_transform(trainDF['text'])

# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, trainDF['label'], test_size=0.3, random_state=2)
# Não conseguimos usar o c-bow300

In [58]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

train_x_text, valid_x_text, train_y_text, valid_y_text = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.3, random_state=2)

In [65]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x_text), maxlen=max_len)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x_text), maxlen=max_len)

In [66]:
train_seq_x.shape

(3420, 25)

# CNN model

In [78]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, validation, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label, epochs=15) #adicionamos epoca
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, validation)

In [84]:
def create_cnn(input_size):
    # Add an Input Layer
    #input_layer = layers.Input((24,))
    input_layer = layers.Input((input_size,1,))

    input_layer = layers.SpatialDropout1D(0.3)(input_layer)

    # Add the word embedding Layer
    #embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=True)(input_layer)
    #embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(input_layer)
    #conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(64, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    #sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['acc', 'mae'])
  
    return model

In [85]:
classifier = create_cnn(max_len)
accuracyCNN = train_model(classifier,train_seq_x, train_y_text, valid_seq_x, valid_y_text, is_neural_net=True)

print ("CNN, Word Embeddings - accuracy",  accuracyCNN)

(None, 1)
Epoch 1/15
107/107 [==============================] - 1s 3ms/step - loss: 0.4885 - acc: 0.5082 - mae: 0.4934
Epoch 2/15
107/107 [==============================] - 0s 3ms/step - loss: 0.5043 - acc: 0.4950 - mae: 0.5055
Epoch 3/15
107/107 [==============================] - 0s 4ms/step - loss: 0.5038 - acc: 0.4962 - mae: 0.5041
Epoch 4/15
107/107 [==============================] - 0s 3ms/step - loss: 0.5044 - acc: 0.4956 - mae: 0.5045
Epoch 5/15
107/107 [==============================] - 0s 4ms/step - loss: 0.5044 - acc: 0.4956 - mae: 0.5045
Epoch 6/15
107/107 [==============================] - 0s 4ms/step - loss: 0.5038 - acc: 0.4962 - mae: 0.5039
Epoch 7/15
107/107 [==============================] - 0s 4ms/step - loss: 0.5035 - acc: 0.4965 - mae: 0.5036
Epoch 8/15
107/107 [==============================] - 0s 4ms/step - loss: 0.5029 - acc: 0.4971 - mae: 0.5031
Epoch 9/15
107/107 [==============================] - 0s 3ms/step - loss: 0.5033 - acc: 0.4965 - mae: 0.5034
Epoch 10/